Data science : Homework 3

6330166021 Nuttapol Onton

In [50]:
# import library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, f1_score

1. Load "ModifiedEdibleMushroom.csv" data from the link below (note: this data set has been preliminarily prepared.).

In [54]:
# !wget "https://github.com/kaopanboonyuen/Python-Data-Science/raw/master/Dataset/hed2020_dataset.csv"
df = pd.read_csv('hed2020_dataset.csv')
print(df.shape)
print('na in gill-size :', df['gill-size'].isnull().sum())

(5824, 24)
na in gill-size : 121


2. Drop rows where the target (label) variable is missing.

In [55]:
df.dropna(axis = 0, subset=['label'], inplace = True)
df.shape
    

(5764, 24)

3. Drop the following variables:

'id','gill-attachment', 'gill-spacing', 'gill-size','gill-color-rate',
'stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring-rate','stalk-color-below-ring-rate','veil-color-rate','veil-type'

In [56]:
drop_list = ['id','gill-attachment', 'gill-spacing', 'gill-size','gill-color-rate','stalk-root', 'stalk-surface-above-ring', 'stalk-surface-below-ring', 'stalk-color-above-ring-rate','stalk-color-below-ring-rate','veil-color-rate','veil-type']
print("drop_list len :", len(drop_list))
df.drop(axis=1, labels = drop_list, inplace = True)
df.shape

drop_list len : 12


(5764, 12)

4. Examine the number of rows, the number of digits, and whether any are missing.

In [58]:
def printMissingValue(df) :
    col_list = df.columns
    ans = []
    for col in col_list:
        tmp = []
        tmp.append(col)
        tmp.append(': ' + str(df[col].isnull().sum()))
        ans.append(tmp)
    print("Number of rows that have missing value in each comlumn")
    for row in ans:
        print("".join(word.ljust(20) for word in row))
printMissingValue(df)
print(df.shape)

Number of rows that have missing value in each comlumn
label               : 0                 
cap-shape           : 0                 
cap-surface         : 27                
bruises             : 99                
odor                : 99                
stalk-shape         : 121               
ring-number         : 62                
ring-type           : 62                
spore-print-color   : 56                
population          : 56                
habitat             : 31                
cap-color-rate      : 27                
(5764, 12)


5. Fill missing values by adding the mean for numeric variables and the mode for nominal variables.

In [59]:
fill_with_mean = df.select_dtypes('number')
fill_with_mode = df.select_dtypes('object')
df[fill_with_mean.columns] = fill_with_mean.fillna(fill_with_mean.mean())
df[fill_with_mode.columns] = fill_with_mode.fillna(fill_with_mode.agg(lambda x: x.mode().values[0]))
printMissingValue(df)

Number of rows that have missing value in each comlumn
label               : 0                 
cap-shape           : 0                 
cap-surface         : 0                 
bruises             : 0                 
odor                : 0                 
stalk-shape         : 0                 
ring-number         : 0                 
ring-type           : 0                 
spore-print-color   : 0                 
population          : 0                 
habitat             : 0                 
cap-color-rate      : 0                 


6. Convert the label variable e (edible) to 1 and p (poisonous) to 0 and check the quantity. class0: class1

In [60]:
df["label"] = np.where(df["label"] == "e", 1, 0)
print(df.shape[0] - df['label'].sum(), ':' , df['label'].sum())

3660 : 2104


7. Convert the nominal variable to numeric using a dummy code with drop_first = True.

In [61]:
nominal_col = df.select_dtypes('object').columns
dummy_df = pd.get_dummies(df[nominal_col], drop_first=True) 
df = pd.concat([df, dummy_df], axis=1)
df = df.drop(nominal_col, axis=1)
df.shape

(5764, 43)

8. Split train/test with 20% test, stratify, and seed = 2020.

In [65]:
input_data = df.drop("label", axis = 1).values.tolist()
output_data = df["label"].values.tolist()
x_train, x_test, y_train, y_test = train_test_split(input_data, output_data, stratify = output_data, test_size=0.20, random_state = 2020)
print([len(x_train), len(x_test)])

[4611, 1153]


9. Create a Random Forest with GridSearch on training data with 5 CV.

In [68]:
random_forest = RandomForestClassifier()

param_grid = { 
    'criterion' :['gini', 'entropy'],
    'min_samples_leaf': [2,5,10],
    'max_depth': [2,3,6],
    'n_estimators': [100, 200],
    'random_state': [2020]
}

model = GridSearchCV(estimator=random_forest, param_grid=param_grid, cv= 5, scoring='accuracy')
model.fit(x_train, y_train)
print(model.best_params_)

{'criterion': 'gini', 'max_depth': 6, 'min_samples_leaf': 2, 'n_estimators': 100, 'random_state': 2020}


10. Predict the testing data set with confusion_matrix and classification_report.

In [69]:
y_pred = model.predict(x_test)

print("Confusion matrix")
print(confusion_matrix(y_true=y_test, y_pred = y_pred))
print('----------------------------------------------------')
print("Classification report")
print(classification_report(y_test, y_pred))
print('----------------------------------------------------')
print("F1 Score :", f1_score(y_test, y_pred))

Confusion matrix
[[731   1]
 [  5 416]]
----------------------------------------------------
Classification report
              precision    recall  f1-score   support

           0       0.99      1.00      1.00       732
           1       1.00      0.99      0.99       421

    accuracy                           0.99      1153
   macro avg       1.00      0.99      0.99      1153
weighted avg       0.99      0.99      0.99      1153

----------------------------------------------------
F1 Score : 0.9928400954653938
